In [1]:
import qualreas as qr
import json

In [2]:
alg_path = "/Users/alfredreich/Documents/Github/repo/qualreas/Algebras/"
json_file = "/Users/alfredreich/Documents/Github/repo/qualreas/Misc/rcc8_example.json"
json_file_noname = "/Users/alfredreich/Documents/Github/repo/qualreas/Misc/rcc8_example_NONAME.json"

In [3]:
!cat {json_file}

{
    "name": "RCC8 Example 1",
    "algebra": "RCC8Algebra",
    "abbreviations": {"?": "DC|EC|TPP|TPPI|PO|EQ|NTPP|NTPPI"},
    "nodes": {
        "h1": ["House1", "Region"],
        "h2": ["House2", "Region"],
        "p1": ["Property1", "Region"],
        "p2": ["Property2", "Region"],
        "rd": ["Road", "Region"]
    },
    "edges": [
        ["h1", "h2", "DC"],
        ["h1", "p1", "TPP|NTPP"],
        ["h1", "p2", "DC|EC"],
        ["h1", "rd", "EC"],
        ["h2", "p1", "DC|EC"],
        ["h2", "p2", "NTPP"],
        ["h2", "rd", "EC"],
        ["p1", "p2", "DC|EC"],
        ["rd", "p1", "?"],
        ["rd", "p2", "?"]
    ]
}

In [4]:
net0 = qr.Network(algebra_path=alg_path, json_file_name=json_file)

In [5]:
net0.summary()


RCC8 Example 1: 5 nodes, 25 edges
  Algebra: RCC8Algebra
  House1:
    => House1: EQ
    => House2: DC
    => Property1: NTPP|TPP
    => Property2: DC|EC
    => Road: EC
  House2:
    => House2: EQ
    => House1: DC
    => Property1: DC|EC
    => Property2: NTPP
    => Road: EC
  Property1:
    => Property1: EQ
    => House1: NTPPI|TPPI
    => House2: DC|EC
    => Property2: DC|EC
    => Road: DC|EC|EQ|NTPP|NTPPI|PO|TPP|TPPI
  Property2:
    => Property2: EQ
    => House1: DC|EC
    => House2: NTPPI
    => Property1: DC|EC
    => Road: DC|EC|EQ|NTPP|NTPPI|PO|TPP|TPPI
  Road:
    => Road: EQ
    => House1: EC
    => House2: EC
    => Property1: DC|EC|EQ|NTPP|NTPPI|PO|TPP|TPPI
    => Property2: DC|EC|EQ|NTPP|NTPPI|PO|TPP|TPPI


In [6]:
net0.propagate()
net0.summary()


RCC8 Example 1: 5 nodes, 25 edges
  Algebra: RCC8Algebra
  House1:
    => House1: EQ
    => House2: DC
    => Property1: NTPP|TPP
    => Property2: DC|EC
    => Road: EC
  House2:
    => House2: EQ
    => House1: DC
    => Property1: DC
    => Property2: NTPP
    => Road: EC
  Property1:
    => Property1: EQ
    => House1: NTPPI|TPPI
    => House2: DC
    => Property2: DC|EC
    => Road: EC|PO
  Property2:
    => Property2: EQ
    => House1: DC|EC
    => House2: NTPPI
    => Property1: DC|EC
    => Road: PO|TPPI
  Road:
    => Road: EQ
    => House1: EC
    => House2: EC
    => Property1: EC|PO
    => Property2: PO|TPP


In [7]:
json_string = """{
    "name": "RCC8 Example 1",
    "algebra": "RCC8Algebra",
    "abbreviations": {"?": "DC|EC|TPP|TPPI|PO|EQ|NTPP|NTPPI"},
    "nodes": {
        "h1": ["House1", "Region"],
        "h2": ["House2", "Region"],
        "p1": ["Property1", "Region"],
        "p2": ["Property2", "Region"],
        "rd": ["Road", "Region"]
    },
    "edges": [
        ["h1", "h2", "DC"],
        ["h1", "p1", "TPP|NTPP"],
        ["h1", "p2", "DC|EC"],
        ["h1", "rd", "EC"],
        ["h2", "p1", "DC|EC"],
        ["h2", "p2", "NTPP"],
        ["h2", "rd", "EC"],
        ["p1", "p2", "DC|EC"],
        ["rd", "p1", "?"],
        ["rd", "p2", "?"]
    ]
}"""

In [9]:
!cat {json_file_noname}

{
    "algebra": "RCC8Algebra",
    "abbreviations": {"?": "DC|EC|TPP|TPPI|PO|EQ|NTPP|NTPPI"},
    "nodes": {
        "h1": ["House1", "Region"],
        "h2": ["House2", "Region"],
        "p1": ["Property1", "Region"],
        "p2": ["Property2", "Region"],
        "rd": ["Road", "Region"]
    },
    "edges": [
        ["h1", "h2", "DC"],
        ["h1", "p1", "TPP|NTPP"],
        ["h1", "p2", "DC|EC"],
        ["h1", "rd", "EC"],
        ["h2", "p1", "DC|EC"],
        ["h2", "p2", "NTPP"],
        ["h2", "rd", "EC"],
        ["p1", "p2", "DC|EC"],
        ["rd", "p1"],
        ["rd", "p2", "?"]
    ]
}

In [8]:
net1 = qr.Network(algebra_path=alg_path, json_file_name=json_file_noname)
net1.summary()


Network:aMDSWBIA: 5 nodes, 25 edges
  Algebra: RCC8Algebra
  House1:
    => House1: EQ
    => House2: DC
    => Property1: NTPP|TPP
    => Property2: DC|EC
    => Road: EC
  House2:
    => House2: EQ
    => House1: DC
    => Property1: DC|EC
    => Property2: NTPP
    => Road: EC
  Property1:
    => Property1: EQ
    => House1: NTPPI|TPPI
    => House2: DC|EC
    => Property2: DC|EC
    => Road: DC|EC|EQ|NTPP|NTPPI|PO|TPP|TPPI
  Property2:
    => Property2: EQ
    => House1: DC|EC
    => House2: NTPPI
    => Property1: DC|EC
    => Road: DC|EC|EQ|NTPP|NTPPI|PO|TPP|TPPI
  Road:
    => Road: EQ
    => House1: EC
    => House2: EC
    => Property1: DC|EC|EQ|NTPP|NTPPI|PO|TPP|TPPI
    => Property2: DC|EC|EQ|NTPP|NTPPI|PO|TPP|TPPI


In [ ]:
len([1, 2, 3])

In [10]:
print(net0.out_degree)

OutDegreeView({<SpatialObject House1 ['Region']>: 5, <SpatialObject House2 ['Region']>: 5, <SpatialObject Property1 ['Region']>: 5, <SpatialObject Property2 ['Region']>: 5, <SpatialObject Road ['Region']>: 5})

In [11]:
from networkx.readwrite import json_graph

In [21]:
data1 = json_graph.node_link_data(net1)

In [22]:
data1

{'directed': True,
 'multigraph': False,
 'graph': {'name': 'Network:aMDSWBIA'},
 'nodes': [{'id': <SpatialObject House1 ['Region']>},
  {'id': <SpatialObject House2 ['Region']>},
  {'id': <SpatialObject Property1 ['Region']>},
  {'id': <SpatialObject Property2 ['Region']>},
  {'id': <SpatialObject Road ['Region']>}],
 'links': [{'constraint': relset(['EQ']),
   'source': <SpatialObject House1 ['Region']>,
   'target': <SpatialObject House1 ['Region']>},
  {'constraint': relset(['DC']),
   'source': <SpatialObject House1 ['Region']>,
   'target': <SpatialObject House2 ['Region']>},
  {'constraint': relset(['NTPP', 'TPP']),
   'source': <SpatialObject House1 ['Region']>,
   'target': <SpatialObject Property1 ['Region']>},
  {'constraint': relset(['DC', 'EC']),
   'source': <SpatialObject House1 ['Region']>,
   'target': <SpatialObject Property2 ['Region']>},
  {'constraint': relset(['EC']),
   'source': <SpatialObject House1 ['Region']>,
   'target': <SpatialObject Road ['Region']>},
  

In [19]:
import networkx as nx

H = nx.gn_graph(2)

H

In [20]:
H = json_graph.node_link_graph(data1)
H